In [ ]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 

#endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"
endpoint_url = "https://query.wikidata.org/sparql"

query_txt = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?textItem ?kuratorLabel ?textUrl
WHERE
{
  <placeholder>
  ?textItem cpsp:P46 ?kuratorStatement. 
  ?kuratorStatement cpsps:P46 ?kuratorItem. 
  ?kuratorItem rdfs:label ?kuratorLabel.
  ?textItem cpsp:P57 ?urlstatement. 
  ?urlstatement cpsps:P57 ?textUrl. 
}"""

query_img ="""
SELECT ?item ?itemLabel ?image WHERE {
    ?item p:P170 ?statement0;
                wdt:P18 ?image.
    ?statement0 (ps:P170/(wdt:P279*)) wd:Q104884.
    #wdt:P149 wd:Q994776;
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY (?itemLabel)
LIMIT 50
"""

query_graph = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?x ?y
WHERE
{
  ?a cpsp:P2 ?c.
  ?c cpsps:P2 ?d.
  ?a rdfs:label ?x.
  ?d rdfs:label ?y.

}LIMIT 1"""

query_graph2 = """
SELECT ?a ?b ?c
WHERE
{
  ?a rdfs:label ?c
}LIMIT 100"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

"""def get_text(textitem_id):
    q = ""
    if textitem_id:
        q = query_txt.replace("<placeholder>", "cps:"+textitem_id+" cpsp:P46 ?kuratorStatement.")
    else:
        q = query_txt.replace("<placeholder>","")
    results_txt = run_query(endpoint_url, q)
    for item in results_txt["results"]["bindings"]:
        print('Wikibase link: ' + '[' + item['textItem']['value'] + ']' + '(' + item['textItem']['value'] + ')' + '\n')
        print('Kurator: ' + item['kuratorLabel']['value'] + '\n')
        headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
        r = requests.get(item['textUrl']['value'], headers=headers, stream=True)
        text = str(r.text) #changed from r.content
        text = text.removeprefix("<!DOCTYPE html>") #changed from "b'<!DOCTYPE html>"
        print(markdownify(text)) #changed
"""
def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

def fetch_image_by_url(url, headers):
    r = requests.get(url, headers=headers, stream=True)
    if r.status_code == 200:
        im = Image.open(r.raw)
        return im
    if r.status_code == 500:
        return None
    if r.status_code == 403:
        return None
    if r.status_code == 429:
        timeout = get_delay(r.headers['retry-after'])
        print('Timeout {} m {} s'.format(timeout // 60, timeout % 60))
        time.sleep(timeout)
        fetch_image_by_url(url, headers)

def get_img(partOfItem_id):
    print("")
    q = ""
"""
    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")
    results_img = run_query(endpoint_url, q)

    print('<div class="column-page">')

    for item in results_img["results"]["bindings"]:    
      #print(item)
      #print('Wikibase link: ' + '[' + item['imgItem']['value'] + ']' + '(' + item['imgItem']['value'] + ')' + '\n')
      #print('Title: ' + item['itemLabel']['value'] + '\n')
      #print('Year: ' + item['publishDate']['value'] + '\n')
      #print('Description: ' + html.unescape(item['itemDescr']['value']) + '\n')

      # get image from image URL and display resized version


      image_url=item['image']['value']
      #headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
      #im = fetch_image_by_url(image_url, headers)
      #im.thumbnail((500, 500), Image.Resampling.LANCZOS)
      #display(im)
      onclickopen = f"openPopover('{image_url}')"
      onclickclose = f"closePopover('{image_url}')"
      print(f'<input type="image" class="img_square" src="{image_url}" popovertarget="{image_url}" onclick="{onclickopen}"/>')
      #print("<div class='fullimage'>")
      print(f'<img popover class="img_popover" src="{image_url}" id="{image_url}" onclick="{onclickclose}" style="display: none;">')    
      #print(f'<div class="close-btn" id="close_button_{image_url} onclick="{onclickclose}" style="display:none;"/></div>')
    print('</div>')"""
print("<link rel='stylesheet' href='./section-fullimg.css'>")
print("""
          
::: {.slideshow}
  <ul class="slides"><li class="slide is-active"><div class="cover" style="background-image: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/162656/unsplash_nature6.jpg);"></div></li><li class="slide"><div class="cover" style="background-image: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/162656/unsplash_nature1.jpg);"></div></li><li class="slide"><div class="cover" style="background-image: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/162656/unsplash_nature6.jpg);"></div></li></ul><div class="arrows"><button class="arrow arrow-prev"></button><button class="arrow arrow-next"></button></div>
:::
::: {.main}
  <div class="container"><p>Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla tempus eleifend tincidunt. Integer lacus orci, ullamcorper vel mi at, vestibulum ultrices quam. Pellentesque dui massa, fringilla id dolor id, imperdiet luctus nisl. Nam vehicula facilisis quam quis vehicula. Sed tincidunt massa nibh, eget varius dui scelerisque in. Donec urna tortor, commodo quis finibus non, vulputate sit amet nisl. Integer consectetur convallis sapien vel rutrum.</p><p>Suspendisse commodo odio magna, tristique semper velit pretium sed. Integer laoreet vestibulum turpis, hendrerit imperdiet nibh egestas at. Etiam eu porttitor magna. Aliquam erat volutpat. Nulla euismod nulla at purusdignissim, vitae mollis elit mollis. Fusce blandit massa sit amet ligula maximus tincidunt. Nullam eu metus vitaenisi ullamcorper varius vitae ac mi.</p></div>
:::


<script>
const slideshow = document.querySelector(".slideshow");
const list = document.querySelector(".slideshow .slides");
const btns = document.querySelectorAll(".slideshow .arrows .arrow");
const prevBtn = document.querySelector(".slideshow .arrow-prev");
const prevBtnChild = document.querySelector(".slideshow .arrow-prev span");
const nextBtn = document.querySelector(".slideshow .arrow-next");
const nextBtnChild = document.querySelector(".slideshow .arrow-next span");
const main = document.querySelector("main");
const autoplayInterval = parseInt(slideshow.dataset.autoplayInterval) || 4000;
const isActive = "is-active";
const isVisible = "is-visible";
let intervalID;

window.addEventListener("load", () => {
  changeSlide();
  autoPlay();
  stopAutoPlayOnHover();
  customizeArrows();
});

function changeSlide() {
  for (const btn of btns) {
    btn.addEventListener("click", e => {
      const activeSlide = document.querySelector(".slide.is-active");
      activeSlide.classList.remove(isActive);
      if (e.currentTarget === nextBtn) {
        activeSlide.nextElementSibling
          ? activeSlide.nextElementSibling.classList.add(isActive)
          : list.firstElementChild.classList.add(isActive);
      } else {
        activeSlide.previousElementSibling
          ? activeSlide.previousElementSibling.classList.add(isActive)
          : list.lastElementChild.classList.add(isActive);
      }
    });
  }
}

function autoPlay() {
  if (slideshow.dataset.autoplay === "true") {
    intervalID = setInterval(() => {
      nextBtn.click();
    }, autoplayInterval);
  }
}

function stopAutoPlayOnHover() {
  if (
    slideshow.dataset.autoplay === "true" &&
    slideshow.dataset.stopAutoplayOnHover === "true"
  ) {
    slideshow.addEventListener("mouseenter", () => {
      clearInterval(intervalID);
    });

    slideshow.addEventListener("touchstart", () => {
      clearInterval(intervalID);
    });
  }
}

function customizeArrows() {
  slideshow.addEventListener("mousemove", mousemoveHandler);
  slideshow.addEventListener("touchmove", mousemoveHandler);
  slideshow.addEventListener("mouseleave", mouseleaveHandler);
  main.addEventListener("touchstart", mouseleaveHandler);
}

function mousemoveHandler(e) {
  const width = this.offsetWidth;
  const xPos = e.pageX;
  const yPos = e.pageY;
  const xPos2 = e.pageX - nextBtn.offsetLeft - nextBtnChild.offsetWidth;

  if (xPos > width / 2) {
    prevBtn.classList.remove(isVisible);
    nextBtn.classList.add(isVisible);

    nextBtnChild.style.left = `${xPos2}px`;
    nextBtnChild.style.top = `${yPos}px`;
  } else {
    nextBtn.classList.remove(isVisible);
    prevBtn.classList.add(isVisible);

    prevBtnChild.style.left = `${xPos}px`;
    prevBtnChild.style.top = `${yPos}px`;
  }
}

//function mouseleaveHandler() {
 // prevBtn.classList.remove(isVisible);
 // nextBtn.classList.remove(isVisible);
//}
</script>
          """)

# """
# def get_graph():
#     import VizKG.visualize as vkg
#     results_graph1 = run_query(endpoint_url, query_graph)
#     #print(results_graph1)
#     #print('---')
#     results_graph2 = run_query(endpoint_url, query_graph2)
#     #print(results_graph2)

#     chart = vkg(sparql_query=query_graph2, sparql_service_url=endpoint_url, chart='wordcloud')
#     chart.plot()
# """

print(
"""
Learned to row a boat away with just one finger<br>
Cast but prayers into the bow<br>
I hope I sink her<br>
Watch me fall asleep in a bag of rope<br>
If the storm comes back, cut my throat<br>
Calm the storm with all I must<br>
I must a-whimper<br>
<br>
Cause I been holding out this hand<br>
*I been holding out this hand*<br>
Old sock money bank<br>
I spent all that I had<br>
Now we go round and around and around<br>
Oh, we go around and around and around<br>
<br>
I cut the storm with just a stick to watch it splinter<br>
Now I'm begging ??? to make a home for winter<br>
Drew my children in the snow <br>
And spilled my blood to watch them grow<br>
Cast me off this boat<br>
It's me who sinks her<br>
<br>
Cause I been holding out this hand<br>
*I been holding out this hand*<br>
Old sock money bank<br>
I spent all that I had<br>
Now we go round and around and around<br>
Oh, we go around and around and around<br>
Oh, we go around and around and around<br>
Oh, we go around and around and around<br>
"""
)

<link rel='stylesheet' href='./section-fullimg.css'>

          
::: {.slideshow}
  <ul class="slides"><li class="slide is-active"><div class="cover" style="background-image: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/162656/unsplash_nature6.jpg);"></div></li><li class="slide"><div class="cover" style="background-image: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/162656/unsplash_nature1.jpg);"></div></li><li class="slide"><div class="cover" style="background-image: url(https://s3-us-west-2.amazonaws.com/s.cdpn.io/162656/unsplash_nature6.jpg);"></div></li></ul><div class="arrows"><button class="arrow arrow-prev"></button><button class="arrow arrow-next"></button></div>
:::
::: {.main}
  <div class="container"><p>Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla tempus eleifend tincidunt. Integer lacus orci, ullamcorper vel mi at, vestibulum ultrices quam. Pellentesque dui massa, fringilla id dolor id, imperdiet luctus nisl. Nam vehicula facilisis quam quis vehicu

In [210]:
#get_text("Q209")

In [211]:
get_img("")

<hr>

In [212]:
#get_graph()